In [4]:
import sys

sys.path.append('/Users/algo/Desktop/Algo_Trading/ShoonyaApi')


import api_helper

import configparser
from api_helper import ShoonyaApiPy

import pandas as pd
import time

from time import sleep
from pytz import timezone 
import datetime as dt 

import numpy as np
from datetime import datetime


#login

config = configparser.ConfigParser()
config.read('credential.ini')
user=config['finvasia']['user']
u_pwd=config['finvasia']['u_pwd']
factor2=config['finvasia']['factor2']
vc=config['finvasia']['vc']
app_key= config['finvasia']['app_key']
imei= config['finvasia']['imei']


In [5]:
global master_contract, api

In [6]:
token = 'B6D32I442C4F743D5BVX6E76DPSBK732'

In [7]:
import pyotp
pyotp.TOTP(token).now()

'686320'

In [8]:
u_pwd

'Lombard@1234'

In [9]:
while dt.datetime.now(timezone("Asia/Kolkata")).time() < dt.time(9,20,0):
    sleep(1)
    continue
api = ShoonyaApiPy()
#credentials by login 
ret = api.login(userid=user, password=u_pwd, twoFA=pyotp.TOTP(token).now(), vendor_code=vc,
api_secret=app_key, imei=imei)


In [10]:
master_contract = pd.read_csv('https://shoonya.finvasia.com/NFO_symbols.txt.zip',compression='zip', engine='python',delimiter=',')
master_contract['Expiry'] = pd.to_datetime(master_contract['Expiry'])
master_contract['StrikePrice'] = master_contract['StrikePrice'].astype(float)
master_contract.sort_values('Expiry',inplace=True)
master_contract.reset_index(drop=True, inplace=True)


In [11]:

def get_instrument(Symbol,strike_price,optiontype,expiry_offset):
    #to get a intstrument token from the master contract downloaded from shoonya website
    return(master_contract[(master_contract['Symbol']==Symbol) & (master_contract['OptionType']==optiontype) & (master_contract['StrikePrice']==strike_price)].iloc[expiry_offset])

def get_atm_strike():
    bnspot_token=api.searchscrip(exchange='NSE',searchtext='Nifty bank')['values'][0]['token']
    while True:
        bnflp=float(api.get_quotes(exchange='NSE', token=bnspot_token)['lp'])
        if bnflp!=None:
            break
    atmprice=round(bnflp/base)*base
    return atmprice


def place_order(BS,tradingsymbol,quantity,product_type='I',price_type='MKT',exchange='NFO',price=0,trigger_price=None):
    order_place=api.place_order(buy_or_sell=BS, product_type=product_type,
                        exchange=exchange, tradingsymbol=tradingsymbol, 
                        quantity=quantity, discloseqty=0,price_type=price_type,price=price,trigger_price=trigger_price) #M for NRML AND I For intraday in product type
    print(order_place)
    return order_place['norenordno']

def stop_loss_order(qty,tradingsymbol,price,SL):
    stop_price=round((1+SL/100)*price,1)
    price=stop_price+2
    trigger_price=stop_price
    stop_loss_orderid=place_order(BS='B',tradingsymbol=tradingsymbol,quantity=qty,price_type='SL-LMT',price=price,trigger_price=trigger_price) 
    return stop_loss_orderid     



def single_order_history(orderid,req):
    ''''stat','norenordno', 'uid', 'src_uid', 'actid', 'exch', 'tsym', 'q 'trantyty',pe', 'prctyp', 'ret', 'rejby', 'kidid',
       'pan', 'ordersource', 'token', 'pp', 'ls', 'ti', 'prc', 'dscqty', 'prd', 'status', 'rpt', 'ordenttm', 'norentm', 'rejreason','exch_tm'''
    #this required to made to avoid unnecesary making a lof of Dataframes
    #dl=pd.DataFrame(api.single_order_history(orderid))
    #sleep(1)
    #return dl[req].iloc[0]

    while True: 
        json_data=api.single_order_history(orderid)
        if json_data!=None:
            break

    for id in json_data:
        value_return=id[req]
        break

    return value_return
        

In [12]:
feed_opened = False
feedJson = {}
counter = 0
def event_handler_order_update(order):
    counter = counter + 1
    print(f"order feed {counter/2}")
    
def event_handler_feed_update(tick_data):
    if 'lp' in tick_data and 'tk' in tick_data :
        timest = datetime.fromtimestamp(int(tick_data['ft'])).isoformat()
        feedJson[tick_data['tk']] = {'ltp': float(tick_data['lp']) , 'tt': timest}

def open_callback():
    global feed_opened
    feed_opened = True


api.start_websocket( order_update_callback=event_handler_order_update,
                     subscribe_callback=event_handler_feed_update, 
                     socket_open_callback=open_callback)

while(feed_opened==False):
    pass

In [13]:
base=100
symbol="BANKNIFTY"
sl=25
qty=25
#while dt.datetime.now(timezone("Asia/Kolkata")).time() < dt.time(9,20,0):
#    sleep(1)

In [ ]:
while (dt.datetime.now(timezone("Asia/Kolkata")).time() < dt.time(15,10,0)):
    atm_strike=get_atm_strike()
    ce_tradingsymbol=get_instrument(symbol,atm_strike,'CE',0)['TradingSymbol']
    pe_tradingsymbol=get_instrument(symbol,atm_strike,'PE',0)['TradingSymbol']


    ce_orderid=place_order('S',ce_tradingsymbol,qty)
    pe_orderid=place_order('S',pe_tradingsymbol,qty)
    sleep(3)
    ce_entry_price=float(single_order_history(ce_orderid,'avgprc'))
    pe_entry_price=float(single_order_history(pe_orderid,'avgprc'))


    ce_token_id = str(get_instrument(symbol,atm_strike,'CE',0)['Token'])
    pe_token_id = str(get_instrument(symbol,atm_strike,'PE',0)['Token'])
    ce_subscribe = 'NFO' + '|' + (ce_token_id)
    pe_subscribe = 'NFO' + '|' + (pe_token_id)
    
    api.subscribe([ce_subscribe, pe_subscribe])
    sleep(3)
    print(feedJson[ce_token_id]['ltp'])
    pe_sl_hit = 0
    ce_sl_hit = 0
    while (dt.datetime.now(timezone("Asia/Kolkata")).time() < dt.time(15,10,0)):
        if(feedJson[ce_token_id]['ltp']) > ce_entry_price * (1+sl/100):
            place_order('B',ce_tradingsymbol,qty)
            pe_sl_hit = 1
            print("CE SL hit")

        if(feedJson[pe_token_id]['ltp']) > pe_entry_price * (1+sl/100):
            place_order('B',pe_tradingsymbol,qty)
            ce_sl_hit = 1
            print("PE SL hit")

#        if(abs((ce_entry_price - feedJson[str(ce_token_id)]['ltp']) +
#                (pe_entry_price - feedJson[str(pe_token_id)]['ltp']))*25 > 500):
        ret = api.get_positions()
        mtm = 0
        for i in ret:
            mtm += float(i['urmtom'])   
        if(abs(mtm) > 500):
            print("Exit condition satisfied")
            print(mtm)          
            if(ce_sl_hit != 1):
                place_order('B',ce_tradingsymbol,qty)
                ce_sl_hit = 1
            if(pe_sl_hit != 1):
                place_order('B',pe_tradingsymbol,qty)
                pe_sl_hit = 1
            api.unsubscribe([ce_subscribe, pe_subscribe])
            break

    if(ce_sl_hit != 1):
        place_order('B',ce_tradingsymbol,qty)
    if(pe_sl_hit != 1):
        place_order('B',pe_tradingsymbol,qty)
    api.unsubscribe([ce_subscribe, pe_subscribe])
    sleep(120)



In [ ]:
if __name__ == "__main__":
11    text = input("Yell something at a mountain: ")
12    print(echo(text))

In [ ]:
#api.get_positions()

In [ ]:
#mtm

In [ ]:
#ret = api.get_positions()
#mtm = 0
#pnl = 0
#for i in ret:
#    mtm += float(i['urmtom'])
#    pnl += float(i['rpnl'])
#    day_m2m = mtm + pnl
#print(f'{day_m2m} is your Daily MTM')